In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Create an array for all the years we're interested in
years = [2011]

In [4]:
# Get list of riding numbers
folderName = "pollresults_resultatsbureau_canada"

# This file contains the provinces and their electoral districts
ridingFile = os.path.join(folderName, "table_tableau11.csv")

In [44]:
ridingList = pd.read_csv(ridingFile)
# We only need the first three columns
ridingList = ridingList.iloc[:, 1:3]
ridingList

,Electoral District Name/Nom de circonscription,Electoral District Number/Num�ro de circonscription
0,Avalon,10001
1,Bonavista--Gander--Grand Falls--Windsor,10002
2,Humber--St. Barbe--Baie Verte,10003
3,Labrador,10004
4,Random--Burin--St. George's,10005
5,St. John's East/St. John's-Est,10006
6,St. John's South--Mount Pearl/St. John's-Sud--...,10007
7,Cardigan,11001
8,Charlottetown,11002
9,Egmont,11003


In [8]:
ridings = [13003, 13008]

In [41]:
def percent_by_polling_district(riding, year):
    fileName = "pollresults_resultatsbureau" + str(riding) + ".csv"
    filePath = os.path.join(folderName, fileName)
    # Load the data
    pollData = pd.read_csv(filePath)
    
    # Get column names, and remove French portions
    colNames = list(pollData.columns.values)
    colNames = [x.split('/')[0] for x in colNames]
    pollData.columns = colNames
    
    # Drop unnecessary columns
    listColDrop = ['Electoral District Name_English',
                   'Electoral District Name_French',
                   'Void Poll Indicator',
                   'No Poll Held Indicator', 
                   'Merge With',
                   'Rejected Ballots for Polling Station',
                   'Political Affiliation Name_French',
                   "Candidate's First Name",
                   "Candidate's Family Name",
                   "Candidate's Middle Name", 
                   'Incumbent Indicator',
                   'Elected Candidate Indicator']
    pollData = pollData.drop(listColDrop, axis=1)
    
    # Strip the polling ID column of whitespace.
    polCol = 'Polling Station Number'
    s = lambda x: str(x).strip(" ")
    pollData[polCol] = pollData[polCol].map(s)
    
    # Create a pivot table of the data by polling district/candidate name
    pollData = pollData.pivot(
        index='Polling Station Number',
        columns='Political Affiliation Name_English',
        values='Candidate Poll Votes Count')
    # Turn the index back into a column
    pollData.reset_index(level=0, inplace=True)
    
    # Strip the letters off polling stations since the geospatial data
    #  does not include these letters.
    stripCharacters = "ABCDEFG"
    s = lambda x: str(x).strip(stripCharacters)
    statCol = 'Polling Station Number'
    pollData[statCol] = pollData[statCol].map(s)
    
    # Merge polling stations
    pollData = pollData.groupby('Polling Station Number').sum()
    pollData.reset_index(level=0, inplace=True)
    
    # Get the vote totals
    pollData['Vote Totals'] = pollData.sum(axis=1, numeric_only=True)

    # Calculate the percent for each

    # Grab the data we want converted to a percent
    numColsPollData = len(pollData.columns)
    pollDataPercent = pollData.iloc[:, range(1, numColsPollData-1)].copy()

    # Divide it by the total votes for each polling district
    pollDataPercent = pollDataPercent.div(pollData['Vote Totals'], axis=0)

    pollDataPercent = np.round(pollDataPercent*100, decimals=2)

    # Rename columns
    colNames = list(pollDataPercent.columns.values)
    colNames = [x + " (%)" for x in colNames]
    pollDataPercent.columns = colNames

    # Merge it with the original data set
    pollData = pd.concat([pollData, pollDataPercent], axis=1)
    return pollData

In [42]:
for riding in ridings:
    for year in years:
        pollData = percent_by_polling_district(riding, year)

In [43]:
pollData

Political Affiliation Name_English,Polling Station Number,CHP Canada,Conservative,Green Party,Liberal,NDP-New Democratic Party,Vote Totals,CHP Canada (%),Conservative (%),Green Party (%),Liberal (%),NDP-New Democratic Party (%)
0,1,3,16,2,10,30,61,4.92,26.23,3.28,16.39,49.18
1,10,1,102,8,45,66,222,0.45,45.95,3.60,20.27,29.73
2,10-1,2,88,6,32,58,186,1.08,47.31,3.23,17.20,31.18
3,100,2,74,2,18,21,117,1.71,63.25,1.71,15.38,17.95
4,101,3,164,14,32,55,268,1.12,61.19,5.22,11.94,20.52
5,102,2,77,7,12,25,123,1.63,62.60,5.69,9.76,20.33
6,103,0,76,6,18,36,136,0.00,55.88,4.41,13.24,26.47
7,104,7,140,11,18,35,211,3.32,66.35,5.21,8.53,16.59
8,105,3,125,7,14,38,187,1.60,66.84,3.74,7.49,20.32
9,106,2,146,3,20,43,214,0.93,68.22,1.40,9.35,20.09
